In [1]:
#General libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from IPython.display import clear_output
import time

#Own code
from utils.data_utils import create_data, create_dgp_data, transformation, standardize
from utils.tvp_models import TVPVARModel

# Suppress scientific notation in numpy
np.set_printoptions(suppress=True)

In [2]:
train = 200

M = 3
p = 4
T = train - p
k = M*(M*p+1)

print("k: " + str(k))

np.random.seed(12345)

k: 39


In [3]:
ds = pd.read_csv("data/fred_qd.csv")

In [4]:
transform=True

gdp = transformation(ds["GDPC1"].iloc[2:].to_numpy(), 5, transform)
cpi = transformation(ds["CPIAUCSL"].iloc[2:].to_numpy(), 6, transform)
fedfund = transformation(ds["FEDFUNDS"].iloc[2:].to_numpy(), 2, transform)
compi = transformation(ds["PPIACO"].iloc[2:].to_numpy(), 6, transform)
borrowings = transformation(ds["TOTRESNS"].iloc[2:].to_numpy(), 6, transform)
sp500 = transformation(ds["S&P 500"].iloc[2:].to_numpy(), 5, transform)
m2 = transformation(ds["M2REAL"].iloc[2:].to_numpy(), 5, transform)

series_total = [gdp, cpi, fedfund]
#series_total = [gdp, cpi, fedfund, compi, borrowings , sp500, m2]

In [5]:
standardized_series = standardize(series_total, train)

In [8]:
complete_T = cpi.shape[0]
X_complete, y_complete = create_data(standardized_series, complete_T, complete_T-p, M, p, k)

In [ ]:
import os

def calculate_derivative (parameter_value):
    error = np.sqrt(1.1e-16)
    derivative = (calculate_msfe(X_complete, y_complete, p, prior='lasso', prior_parameters={'lambda_param':parameter_value+error})[0] - calculate_msfe(X_complete, y_complete, p, prior='lasso', prior_parameters={'lambda_param':parameter_value-error})[0])/(2*error)
    print('lambda: ' + str(parameter_value) + " is done!" + "\n" + "process ID: " + str(os.getpid()))
    return derivative

In [ ]:
%%time
import os
from multiprocessing import Pool, Array

lambda_values = np.arange(50,51,0.20)

if __name__ == '__main__':
    
    pool = Pool()
    derivatives = pool.map(calculate_derivative, lambda_values)

In [ ]:
import pickle

with open('derivatives.pkl', 'wb') as f:
    pickle.dump(derivatives, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
def calculate_derivative_insample (parameter_value):
    error = np.sqrt(1.1e-16)
    derivative = (insample_msfe(X_complete, y_complete, p, prior='lasso', prior_parameters={'lambda_param':parameter_value+error}) - insample_msfe(X_complete, y_complete, p, prior='lasso', prior_parameters={'lambda_param':parameter_value-error}))/(2*error)
    return derivative

In [ ]:
%%time
import os
from multiprocessing import Pool, Array
from functools import partial

lambda_values = np.arange(50,51,0.025)
print(lambda_values.shape[0])

if __name__ == '__main__':
    
    pool = Pool()
    derivatives = pool.map(calculate_derivative_insample, lambda_values)

In [ ]:
plt.plot(lambda_values, derivatives)

In [ ]:
def calculate_derivative_point_gdp(parameter_value):
    error = np.sqrt(1.1e-16)
    derivative = (point_prediction_full(X_complete, y_complete, p, 'lasso', {'lambda_param':parameter_value+error})[0] - point_prediction_full(X_complete, y_complete, p, 'lasso', {'lambda_param':parameter_value-error})[0])/(2*error)
    return derivative

In [ ]:
calculate_derivative_point_gdp(50)

In [ ]:
%%time
import os
from multiprocessing import Pool, Array
from functools import partial

lambda_values = np.arange(30,400,0.50)
print(lambda_values.shape[0])

if __name__ == '__main__':
    
    pool = Pool()
    derivatives = pool.map(calculate_derivative_point_gdp, lambda_values)

In [ ]:
plt.plot(lambda_values, derivatives)